In [1]:
import os
import mne
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 6)
mpl.rcParams['figure.dpi'] = 120

%matplotlib tk

In [ ]:
# Constants
sampling_rate = 1000
data_folder = './task_data'
subject_id = "032"

# Load single subject data for exploration
parquet_file = f"sub-{subject_id}_task-memory_ecg_task.parquet"
data = pd.read_parquet(os.path.join(data_folder, parquet_file))
ecg_signal = data['ecg_nk']
raw_signal = data['ecg_raw']

print(f"✅ Data loaded successfully")
print(f"   Shape: {data.shape}")
print(f"   Columns: {list(data.columns)}")
data

✅ Data loaded successfully
   Shape: (7395320, 4)
   Columns: ['time', 'ecg_raw', 'ecg_mod', 'ecg_nk']


,time,ecg_raw,ecg_mod,ecg_nk
0,0.000,-0.136761,0.282053,-0.136761
1,0.001,-0.140894,0.251851,-0.140894
2,0.002,-0.143575,0.232259,-0.143575
3,0.003,-0.144166,0.227942,-0.144166
4,0.004,-0.146780,0.208840,-0.146780
...,...,...,...,...
7395315,7395.315,0.153893,0.582470,0.153893
7395316,7395.316,0.164478,0.659819,0.164478
7395317,7395.317,0.170650,0.704917,0.170650
7395318,7395.318,0.173162,0.723276,0.173162


In [3]:
events_file = f"sub-{subject_id}_task-memory_events.parquet"
events_data = pd.read_parquet(os.path.join(data_folder, events_file))
events_data


,onset,duration,sample,trial_type,response_time,stim_file,value
0,0.000,NaN,0,STATUS,NaN,NaN,boundary
1,684.496,1.0,684496,control 01/13: listen to digit 1 (first) in 13...,NaN,NaN,500113
2,686.559,1.0,686559,control 02/13: listen to digit 2 in 13 digit s...,NaN,NaN,500213
3,688.587,1.0,688587,control 03/13: listen to digit 3 in 13 digit s...,NaN,NaN,500313
4,690.630,1.0,690630,control 04/13: listen to digit 4 in 13 digit s...,NaN,NaN,500413
...,...,...,...,...,...,...,...
1454,7342.144,1.0,7342144,control 05/09: listen to digit 5 in 9 digit se...,NaN,NaN,500509
1455,7344.203,1.0,7344203,control 06/09: listen to digit 6 in 9 digit se...,NaN,NaN,500609
1456,7346.222,1.0,7346222,control 07/09: listen to digit 7 in 9 digit se...,NaN,NaN,500709
1457,7348.252,1.0,7348252,control 08/09: listen to digit 8 in 9 digit se...,NaN,NaN,500809
